In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
        #    img = augs(image=img)['image']
        
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [12]:
#import pretrainedmodels

In [13]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [16]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [17]:
#model.last_linear.in_features

In [18]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        #x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        #for i in range(len(x)):
        #    transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        #x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [19]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [20]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [21]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [22]:
round(1/9, 6)

0.111111

In [23]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [24]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [25]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [26]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [27]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [28]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [29]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [30]:
np.random.random()

0.44358804003528074

In [31]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from over9000.lookahead import Lookahead
def LookaheadSGD(params, alpha=0.5, k=6, *args, **kwargs):
     sgd = optim.SGD(params, *args, **kwargs)
     return Lookahead(sgd, alpha, k)

In [45]:
def train(args):
    global model
    
    print(args)

    if args.optim == 'Adam':
        #optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
        optimizer = optim.Adam([{'params': model.parameters(), 'initial_lr': args.lr }], lr=args.lr, weight_decay=1e-5)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        #optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-5)
        optimizer = LookaheadSGD(
            [{'params': model.parameters(), 'initial_lr': args.lr }],
            lr=args.lr, momentum=0.9, weight_decay=1e-5)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, args.t_max, eta_min=args.min_lr, last_epoch=args.t_max)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.5:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
                img, targets = mixup(img, targets)
                outputs = model(img)
                loss = mixup_criterion(outputs, targets)
                #loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [46]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model3_se_resnext50_fold1_mixup_cutmix.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 1e-4
args.lrs = 'cosine'
args.t_max = 12
args.factor = 0.1
args.patience = 10
args.min_lr = 1e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 1024
args.val_batch_size = 1024

args.beta = 1.0
args.cutmix_prob = 0.5

In [34]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=1)

(200840, 5)
(200840, 32332)
(160635, 5) (40205, 5)


In [48]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./models/se_resnext50_32x4d/model3_se_resnext50_fold1_mixup_cutmix.pth, exist: True
loading ./models/se_resnext50_32x4d/model3_se_resnext50_fold1_mixup_cutmix.pth...


In [49]:
train(args)

Namespace(backbone='se_resnext50_32x4d', batch_size=1024, beta=1.0, ckp_name='model3_se_resnext50_fold1_mixup_cutmix.pth', cutmix_prob=0.5, factor=0.1, iter_val=200, lr=0.0001, lrs='cosine', min_lr=1e-06, num_epochs=100000, optim='Adam', patience=10, predict=False, t_max=12, val_batch_size=1024)
{'recall': 0.983677, 'recall_grapheme': 0.976234, 'recall_vowel': 0.98942, 'recall_consonant': 0.992822, 'acc_grapheme': 0.974008, 'acc_vowel': 0.991071, 'acc_consonant': 0.990747, 'loss_grapheme': 0.260177, 'loss_vowel': 0.170087, 'loss_consonant': 0.123249}
    1 | 0.000001 | 045056/160635 | 2.0904 | 2.3344 |
val: {'recall': 0.983608, 'recall_grapheme': 0.975824, 'recall_vowel': 0.990264, 'recall_consonant': 0.992522, 'acc_grapheme': 0.974779, 'acc_vowel': 0.991593, 'acc_consonant': 0.990922, 'loss_grapheme': 0.237221, 'loss_vowel': 0.158403, 'loss_consonant': 0.116476}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000003 | 090112/160635 | 3.8444 | 2.4709 |
val: {'recall': 0.983178, 'recall_grapheme': 0.976542, 'recall_vowel': 0.989967, 'recall_consonant': 0.989662, 'acc_grapheme': 0.974879, 'acc_vowel': 0.991543, 'acc_consonant': 0.991046, 'loss_grapheme': 0.333962, 'loss_vowel': 0.22906, 'loss_consonant': 0.156294}
    3 | 0.000008 | 135168/160635 | 2.2165 | 2.1780 |
val: {'recall': 0.983076, 'recall_grapheme': 0.97625, 'recall_vowel': 0.990412, 'recall_consonant': 0.989393, 'acc_grapheme': 0.975028, 'acc_vowel': 0.991469, 'acc_consonant': 0.990474, 'loss_grapheme': 0.190356, 'loss_vowel': 0.100459, 'loss_consonant': 0.084499}
    5 | 0.000015 | 020480/160635 | 3.8531 | 2.4569 |
val: {'recall': 0.982217, 'recall_grapheme': 0.974871, 'recall_vowel': 0.989584, 'recall_consonant': 0.989543, 'acc_grapheme': 0.974207, 'acc_vowel': 0.99122, 'acc_consonant': 0.990747, 'loss_grapheme': 0.309306, 'loss_vowel': 0.219908, 'loss_consonant': 0.150654}
    6 | 0.000026 | 065536/160635 | 3.8297 | 2.

   35 | 0.000015 | 143360/160635 | 2.2455 | 2.2200 |
val: {'recall': 0.984993, 'recall_grapheme': 0.97823, 'recall_vowel': 0.990737, 'recall_consonant': 0.992777, 'acc_grapheme': 0.976073, 'acc_vowel': 0.992513, 'acc_consonant': 0.992091, 'loss_grapheme': 0.247657, 'loss_vowel': 0.18098, 'loss_consonant': 0.126431}
   37 | 0.000026 | 028672/160635 | 3.7019 | 2.3401 |
val: {'recall': 0.984241, 'recall_grapheme': 0.977432, 'recall_vowel': 0.990938, 'recall_consonant': 0.991163, 'acc_grapheme': 0.975476, 'acc_vowel': 0.992364, 'acc_consonant': 0.991742, 'loss_grapheme': 0.250919, 'loss_vowel': 0.175898, 'loss_consonant': 0.120783}
   38 | 0.000038 | 073728/160635 | 3.6996 | 2.3324 |
val: {'recall': 0.983927, 'recall_grapheme': 0.977136, 'recall_vowel': 0.990032, 'recall_consonant': 0.991406, 'acc_grapheme': 0.974978, 'acc_vowel': 0.992339, 'acc_consonant': 0.990897, 'loss_grapheme': 0.269713, 'loss_vowel': 0.184357, 'loss_consonant': 0.125567}
   39 | 0.000050 | 118784/160635 | 3.0159 | 2

   69 | 0.000038 | 036864/160635 | 2.4610 | 2.1686 |
val: {'recall': 0.984814, 'recall_grapheme': 0.978799, 'recall_vowel': 0.991126, 'recall_consonant': 0.990532, 'acc_grapheme': 0.977192, 'acc_vowel': 0.992737, 'acc_consonant': 0.991618, 'loss_grapheme': 0.253304, 'loss_vowel': 0.180456, 'loss_consonant': 0.125874}
   70 | 0.000051 | 081920/160635 | 0.5252 | 1.9787 |
val: {'recall': 0.985653, 'recall_grapheme': 0.979328, 'recall_vowel': 0.991478, 'recall_consonant': 0.99248, 'acc_grapheme': 0.977043, 'acc_vowel': 0.992961, 'acc_consonant': 0.992091, 'loss_grapheme': 0.195625, 'loss_vowel': 0.137956, 'loss_consonant': 0.101491}
   71 | 0.000063 | 126976/160635 | 1.0431 | 2.2177 |
val: {'recall': 0.985487, 'recall_grapheme': 0.980201, 'recall_vowel': 0.991331, 'recall_consonant': 0.990214, 'acc_grapheme': 0.977068, 'acc_vowel': 0.992314, 'acc_consonant': 0.991618, 'loss_grapheme': 0.24609, 'loss_vowel': 0.194368, 'loss_consonant': 0.120942}
   73 | 0.000075 | 012288/160635 | 2.3321 | 2

  102 | 0.000063 | 090112/160635 | 3.7992 | 2.0780 |
val: {'recall': 0.985838, 'recall_grapheme': 0.980367, 'recall_vowel': 0.992233, 'recall_consonant': 0.990388, 'acc_grapheme': 0.978261, 'acc_vowel': 0.99326, 'acc_consonant': 0.992265, 'loss_grapheme': 0.212108, 'loss_vowel': 0.146763, 'loss_consonant': 0.10454}
  103 | 0.000075 | 135168/160635 | 1.2545 | 2.1160 |
val: {'recall': 0.986154, 'recall_grapheme': 0.981112, 'recall_vowel': 0.991653, 'recall_consonant': 0.99074, 'acc_grapheme': 0.979331, 'acc_vowel': 0.992961, 'acc_consonant': 0.992265, 'loss_grapheme': 0.192632, 'loss_vowel': 0.139365, 'loss_consonant': 0.096497}
  105 | 0.000086 | 020480/160635 | 3.0426 | 2.1825 |
val: {'recall': 0.985144, 'recall_grapheme': 0.979303, 'recall_vowel': 0.991517, 'recall_consonant': 0.990453, 'acc_grapheme': 0.976595, 'acc_vowel': 0.99219, 'acc_consonant': 0.991643, 'loss_grapheme': 0.244186, 'loss_vowel': 0.160959, 'loss_consonant': 0.107405}
  106 | 0.000093 | 065536/160635 | 3.2499 | 2.0

  135 | 0.000086 | 143360/160635 | 2.7792 | 1.9964 |
val: {'recall': 0.986377, 'recall_grapheme': 0.979782, 'recall_vowel': 0.992121, 'recall_consonant': 0.993825, 'acc_grapheme': 0.978958, 'acc_vowel': 0.992712, 'acc_consonant': 0.992986, 'loss_grapheme': 0.228357, 'loss_vowel': 0.179679, 'loss_consonant': 0.116876}
  137 | 0.000093 | 028672/160635 | 2.9038 | 2.4141 |
val: {'recall': 0.986627, 'recall_grapheme': 0.981186, 'recall_vowel': 0.991381, 'recall_consonant': 0.992754, 'acc_grapheme': 0.978634, 'acc_vowel': 0.992911, 'acc_consonant': 0.992613, 'loss_grapheme': 0.270124, 'loss_vowel': 0.198079, 'loss_consonant': 0.125416}
  138 | 0.000098 | 073728/160635 | 2.3268 | 2.1209 |
val: {'recall': 0.985957, 'recall_grapheme': 0.980551, 'recall_vowel': 0.990632, 'recall_consonant': 0.992093, 'acc_grapheme': 0.97764, 'acc_vowel': 0.99229, 'acc_consonant': 0.99224, 'loss_grapheme': 0.234382, 'loss_vowel': 0.173528, 'loss_consonant': 0.116617}
  139 | 0.000100 | 118784/160635 | 1.9852 | 2.

  169 | 0.000098 | 036864/160635 | 0.5497 | 1.8971 |
val: {'recall': 0.985981, 'recall_grapheme': 0.979667, 'recall_vowel': 0.99194, 'recall_consonant': 0.992651, 'acc_grapheme': 0.978684, 'acc_vowel': 0.992911, 'acc_consonant': 0.992687, 'loss_grapheme': 0.197685, 'loss_vowel': 0.14456, 'loss_consonant': 0.100354}
  170 | 0.000100 | 081920/160635 | 2.5588 | 2.0958 |
val: {'recall': 0.98582, 'recall_grapheme': 0.980733, 'recall_vowel': 0.992607, 'recall_consonant': 0.989209, 'acc_grapheme': 0.978237, 'acc_vowel': 0.99326, 'acc_consonant': 0.992862, 'loss_grapheme': 0.213107, 'loss_vowel': 0.147968, 'loss_consonant': 0.103087}
  171 | 0.000098 | 126976/160635 | 3.7760 | 2.0841 |
val: {'recall': 0.986159, 'recall_grapheme': 0.981718, 'recall_vowel': 0.991368, 'recall_consonant': 0.989833, 'acc_grapheme': 0.979182, 'acc_vowel': 0.993185, 'acc_consonant': 0.992091, 'loss_grapheme': 0.268548, 'loss_vowel': 0.221224, 'loss_consonant': 0.13798}
  173 | 0.000093 | 012288/160635 | 1.9614 | 1.84

  202 | 0.000098 | 090112/160635 | 1.9797 | 2.0746 |
val: {'recall': 0.987258, 'recall_grapheme': 0.981245, 'recall_vowel': 0.992241, 'recall_consonant': 0.9943, 'acc_grapheme': 0.979729, 'acc_vowel': 0.993185, 'acc_consonant': 0.992936, 'loss_grapheme': 0.303623, 'loss_vowel': 0.227468, 'loss_consonant': 0.143516}
  203 | 0.000093 | 135168/160635 | 2.6338 | 2.0082 |
val: {'recall': 0.987311, 'recall_grapheme': 0.982169, 'recall_vowel': 0.993013, 'recall_consonant': 0.991892, 'acc_grapheme': 0.981594, 'acc_vowel': 0.994055, 'acc_consonant': 0.994031, 'loss_grapheme': 0.189542, 'loss_vowel': 0.153, 'loss_consonant': 0.102781}
  205 | 0.000086 | 020480/160635 | 3.4439 | 2.0573 |
val: {'recall': 0.986924, 'recall_grapheme': 0.981991, 'recall_vowel': 0.993715, 'recall_consonant': 0.989999, 'acc_grapheme': 0.981569, 'acc_vowel': 0.994404, 'acc_consonant': 0.99321, 'loss_grapheme': 0.217566, 'loss_vowel': 0.165105, 'loss_consonant': 0.111523}
  206 | 0.000075 | 065536/160635 | 1.7806 | 1.952

KeyboardInterrupt: 

In [36]:
train(args)

Namespace(backbone='se_resnext50_32x4d', batch_size=1024, beta=1.0, ckp_name='model3_se_resnext50_fold1_mixup_cutmix.pth', cutmix_prob=0.5, factor=0.1, iter_val=200, lr=0.01, lrs='plateau', min_lr=0.0001, num_epochs=100000, optim='SGD', patience=10, predict=False, t_max=12, val_batch_size=1024)
{'recall': 0.040004, 'recall_grapheme': 0.001886, 'recall_vowel': 0.075343, 'recall_consonant': 0.080902, 'acc_grapheme': 0.006168, 'acc_vowel': 0.085064, 'acc_consonant': 0.060192, 'loss_grapheme': 5.120304, 'loss_vowel': 2.479316, 'loss_consonant': 2.24724}


/home/chec/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


    1 | 0.010000 | 045056/160635 | 4.3485 | 5.5337 |
val: {'recall': 0.619079, 'recall_grapheme': 0.304756, 'recall_vowel': 0.922558, 'recall_consonant': 0.944246, 'acc_grapheme': 0.481482, 'acc_vowel': 0.906081, 'acc_consonant': 0.920284, 'loss_grapheme': 2.407548, 'loss_vowel': 0.503989, 'loss_consonant': 0.319169}
** saved
    2 | 0.010000 | 090112/160635 | 5.2682 | 4.5590 |
val: {'recall': 0.827354, 'recall_grapheme': 0.697855, 'recall_vowel': 0.960916, 'recall_consonant': 0.952792, 'acc_grapheme': 0.794677, 'acc_vowel': 0.94446, 'acc_consonant': 0.95411, 'loss_grapheme': 1.24508, 'loss_vowel': 0.414207, 'loss_consonant': 0.254942}
** saved
    3 | 0.010000 | 135168/160635 | 2.5135 | 4.3535 |
val: {'recall': 0.916682, 'recall_grapheme': 0.865008, 'recall_vowel': 0.971118, 'recall_consonant': 0.965595, 'acc_grapheme': 0.875239, 'acc_vowel': 0.963537, 'acc_consonant': 0.965054, 'loss_grapheme': 0.826253, 'loss_vowel': 0.364424, 'loss_consonant': 0.244198}
** saved
    5 | 0.010000 | 

   34 | 0.010000 | 098304/160635 | 3.0032 | 2.9480 |
val: {'recall': 0.974049, 'recall_grapheme': 0.961959, 'recall_vowel': 0.985959, 'recall_consonant': 0.986319, 'acc_grapheme': 0.958985, 'acc_vowel': 0.985872, 'acc_consonant': 0.985599, 'loss_grapheme': 0.438802, 'loss_vowel': 0.241194, 'loss_consonant': 0.158914}
** saved
   35 | 0.010000 | 143360/160635 | 5.1995 | 2.9753 |
val: {'recall': 0.973024, 'recall_grapheme': 0.959212, 'recall_vowel': 0.986018, 'recall_consonant': 0.987653, 'acc_grapheme': 0.957294, 'acc_vowel': 0.985574, 'acc_consonant': 0.985052, 'loss_grapheme': 0.376338, 'loss_vowel': 0.238082, 'loss_consonant': 0.156815}
   37 | 0.010000 | 028672/160635 | 2.2374 | 2.9245 |
val: {'recall': 0.971169, 'recall_grapheme': 0.958371, 'recall_vowel': 0.985123, 'recall_consonant': 0.982811, 'acc_grapheme': 0.955304, 'acc_vowel': 0.986295, 'acc_consonant': 0.98632, 'loss_grapheme': 0.378169, 'loss_vowel': 0.236004, 'loss_consonant': 0.162918}
   38 | 0.010000 | 073728/160635 | 

** saved
   69 | 0.010000 | 036864/160635 | 2.3102 | 2.5358 |
val: {'recall': 0.977795, 'recall_grapheme': 0.967562, 'recall_vowel': 0.988873, 'recall_consonant': 0.987185, 'acc_grapheme': 0.965427, 'acc_vowel': 0.98923, 'acc_consonant': 0.987066, 'loss_grapheme': 0.394783, 'loss_vowel': 0.219057, 'loss_consonant': 0.168694}
   70 | 0.010000 | 081920/160635 | 2.8215 | 2.6148 |
val: {'recall': 0.979946, 'recall_grapheme': 0.971102, 'recall_vowel': 0.988889, 'recall_consonant': 0.988688, 'acc_grapheme': 0.969233, 'acc_vowel': 0.989753, 'acc_consonant': 0.989429, 'loss_grapheme': 0.274274, 'loss_vowel': 0.193647, 'loss_consonant': 0.137678}
** saved
   71 | 0.010000 | 126976/160635 | 2.3870 | 2.4534 |
val: {'recall': 0.980217, 'recall_grapheme': 0.972409, 'recall_vowel': 0.987571, 'recall_consonant': 0.988479, 'acc_grapheme': 0.969332, 'acc_vowel': 0.989404, 'acc_consonant': 0.987464, 'loss_grapheme': 0.352871, 'loss_vowel': 0.213534, 'loss_consonant': 0.149456}
** saved
   73 | 0.010000 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  115 | 0.010000 | 061440/160635 | 2.4583 | 2.3523 |
val: {'recall': 0.981129, 'recall_grapheme': 0.973491, 'recall_vowel': 0.988451, 'recall_consonant': 0.989085, 'acc_grapheme': 0.972168, 'acc_vowel': 0.990499, 'acc_consonant': 0.989976, 'loss_grapheme': 0.313539, 'loss_vowel': 0.200991, 'loss_consonant': 0.144666}
  116 | 0.010000 | 106496/160635 | 1.9569 | 2.4711 |
val: {'recall': 0.982192, 'recall_grapheme': 0.975308, 'recall_vowel': 0.988854, 'recall_consonant': 0.989298, 'acc_grapheme': 0.971745, 'acc_vowel': 0.990349, 'acc_consonant': 0.989951, 'loss_grapheme': 0.275765, 'loss_vowel': 0.169195, 'loss_consonant': 0.113746}
** saved
  117 | 0.010000 | 151552/160635 | 2.0201 | 2.4205 |
val: {'recall': 0.981754, 'recall_grapheme': 0.972246, 'recall_vowel': 0.989643, 'recall_consonant': 0.992881, 'acc_grapheme': 0.97167, 'acc_vowel': 0.990424, 'acc_consonant': 0.988857, 'loss_grapheme': 0.267233, 'loss_vowel': 0.18243, 'loss_consonant': 0.133519}
  119 | 0.010000 | 036864/160635 | 4

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  160 | 0.001000 | 040960/160635 | 2.3495 | 1.9895 |
val: {'recall': 0.983491, 'recall_grapheme': 0.976899, 'recall_vowel': 0.989648, 'recall_consonant': 0.990517, 'acc_grapheme': 0.975351, 'acc_vowel': 0.991444, 'acc_consonant': 0.991096, 'loss_grapheme': 0.189355, 'loss_vowel': 0.116638, 'loss_consonant': 0.091151}
  161 | 0.001000 | 086016/160635 | 1.3966 | 2.2144 |
val: {'recall': 0.983231, 'recall_grapheme': 0.976033, 'recall_vowel': 0.990045, 'recall_consonant': 0.990813, 'acc_grapheme': 0.974779, 'acc_vowel': 0.991593, 'acc_consonant': 0.991195, 'loss_grapheme': 0.235409, 'loss_vowel': 0.160771, 'loss_consonant': 0.114142}
  162 | 0.001000 | 131072/160635 | 2.1251 | 2.2012 |
val: {'recall': 0.982889, 'recall_grapheme': 0.975623, 'recall_vowel': 0.989942, 'recall_consonant': 0.990369, 'acc_grapheme': 0.974555, 'acc_vowel': 0.991643, 'acc_consonant': 0.990922, 'loss_grapheme': 0.233491, 'loss_vowel': 0.161355, 'loss_consonant': 0.113815}
  163 | 0.001000 | 001024/160635 | 3.0853 |

KeyboardInterrupt: 

In [47]:
del model